In [2]:
!pip install pyDOE
!pip install cma


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F10 '''
def F10(X):
    f = bn.F10()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 1300.69 ,random_state=0 , l1_ratio=0.9898, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 69.69 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_10_4000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_10_800Samples.csv').iloc[:,-1]
path = "train_latent_200D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_200D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [6]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [7]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,101):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 100000)

n_obs , dim =  30, 100
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,300])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.1566 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.1566 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.1566 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=874289, Mon Apr 13 17:26:57 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 1.602895434991437e+08 1.0e+00 1.50e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 17:26:57 2020)
final/bestever f-value = 1.602895e+08 1.602895e+08
incumbent solution: [0.90142071 0.90316213 0.27143583 0.68234431 0.39190943 0.61326825
 0.05575508 0.46523461 ...]
std deviations: [0.15042393 0.15044526 0.15050233 0.15045765 0.15034361 0.15035834
 0.15043031 0.15040646 ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=848967, Mon Apr 13 17:26:57 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 1.568751720765490e+08 1.0e+00 1.50e-01  2e-01  2e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.60289543e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=160289543.499144, sigma=1.50e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=160289543.499144, sigma=1.50e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteratio

termination on tolfun=1e-11 (Mon Apr 13 17:26:57 2020)
final/bestever f-value = 1.568752e+08 1.568752e+08
incumbent solution: [0.83093326 0.87736153 0.51461769 0.55469156 0.44780632 0.51593452
 0.03073604 0.421472   ...]
std deviations: [0.15045807 0.15044623 0.15050107 0.15051256 0.15039378 0.15043144
 0.15041076 0.15040468 ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=870434, Mon Apr 13 17:26:57 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 1.603152055866632e+08 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0
    2     34 1.602801343770883e+08 1.0e+00 1.46e-01  1e-01  1e-01 0:00.1
    3     51 1.602420192016157e+08 1.0e+00 1.43e-01  1e-01  1e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.60358656e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.60417375e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   80   1360 -3.846403339927670e+07 1.3e+00 1.94e+00  2e+00  2e+00 0:03.1
  100   1700 -9.616367145655402e+08 1.3e+00 4.66e+00  5e+00  5e+00 0:03.9
  174   2958 -1.362318540824353e+12 1.3e+00 1.63e+02  2e+02  2e+02 0:06.8
termination on tolfacupx=1000.0 (Mon Apr 13 17:27:05 2020)
final/bestever f-value = -1.276809e+12 -1.362319e+12
incumbent solution: [-633.31235793    7.58810771  203.74489276 -334.59470967 -200.24749565
 -119.19431416  997.60923152  175.8424101  ...]
std deviations: [160.0034777  160.39538218 160.21250513 161.0789357  159.02027669
 160.18159626 161.48882868 160.35938454 ...]
Run : 1
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=894796, Mon Apr 13 17:27:05 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 1.602895434991437e+08 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 17:27:05 2020)
final/bestever f-value = 1.602895e+08 1.602895e+08
incumbent solution: [0.20596578 0.3225918  0.736

...

   81   1377 -4.035544106798559e+07 1.3e+00 2.07e+00  2e+00  2e+00 0:03.1
  100   1700 -1.013305765207317e+09 1.3e+00 5.07e+00  5e+00  5e+00 0:03.8
  171   2907 -1.085584890410957e+12 1.4e+00 1.58e+02  2e+02  2e+02 0:06.6
termination on tolfacupx=1000.0 (Mon Apr 13 17:30:47 2020)
final/bestever f-value = -1.067071e+12 -1.085585e+12
incumbent solution: [-378.45191009  177.31752293  575.23774097  -91.1318389  -282.62188964
 -498.89167591  282.25475979   18.21037919 ...]
std deviations: [154.59164922 155.0363834  155.4533977  156.12877587 155.2674252
 154.96239161 156.90667521 155.45175007 ...]
Run : 29
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=906589, Mon Apr 13 17:30:47 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 1.602895434991437e+08 1.0e+00 1.50e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 17:30:47 2020)
final/bestever f-value = 1.602895e+08 1.602895e+08
incumbent solution: [0.01018111 0.73788431 0.728

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.60289543e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=160289543.499144, sigma=1.50e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=160289543.499144, sigma=1.50e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteratio

termination on tolfun=1e-11 (Mon Apr 13 17:30:47 2020)
final/bestever f-value = 1.568752e+08 1.568752e+08
incumbent solution: [0.10399788 0.79825855 0.76261484 0.97144081 0.18056769 0.33879061
 0.23595291 0.53939713 ...]
std deviations: [0.15051044 0.15050443 0.15052039 0.15052803 0.15054302 0.1505372
 0.15052324 0.1505211  ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=911612, Mon Apr 13 17:30:47 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 1.602240085060248e+08 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0
    2     34 1.601826217117914e+08 1.0e+00 1.46e-01  1e-01  1e-01 0:00.1
    3     51 1.601207299555679e+08 1.0e+00 1.43e-01  1e-01  1e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.60327501e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.60229313e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   81   1377 -2.204731770903715e+08 1.3e+00 2.81e+00  3e+00  3e+00 0:03.1
  100   1700 -1.966044381834610e+09 1.3e+00 6.78e+00  7e+00  7e+00 0:03.9
  170   2890 -1.494027678268475e+12 1.3e+00 1.57e+02  2e+02  2e+02 0:06.6
termination on tolfacupx=1000.0 (Mon Apr 13 17:30:55 2020)
final/bestever f-value = -1.444084e+12 -1.494028e+12
incumbent solution: [ -428.85176498  -139.97932132   842.05979522  -205.10962241
  -581.11797211 -1053.11707609   784.92906554   421.61328777 ...]
std deviations: [155.11785225 154.89202095 155.01624158 155.07862199 152.97665653
 155.7826029  156.03533626 155.95033412 ...]


In [8]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F10_200_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F10(X_Values.iloc[i,:100])
    SVM_Fun [i] = F10(X_Values.iloc[i,100:200])
    ELN_Fun [i] = F10(X_Values.iloc[i,200:300])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([44683268.7395525]), count=array([1])) 2834128.0013840357
SVM
ModeResult(mode=array([45038975.98065136]), count=array([1])) 3240420.9665371613
ELN
ModeResult(mode=array([1.60100367e+12]), count=array([1])) 467301585230.57336
